### case enem

**problema:**

In [1]:
# importar as bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# ler as bases de dados

dicionario_colunas = pd.read_excel('dicionario_colunas_enem.xlsx')
base_enem = pd.read_excel('enem.xlsx')

In [8]:
# checar o número de linhas e de colunas de cada dataframe
pd.DataFrame({
    'base': ['base_enem', 'dicionario_colunas'],
    'n_colunas': [len(base_enem.columns), len(dicionario_colunas.columns)],
    'n_linhas': [len(base_enem), len(dicionario_colunas)]
})


,base,n_colunas,n_linhas
0,base_enem,29,18306
1,dicionario_colunas,6,140


In [9]:
dicionario_colunas.head()

,DICIONÁRIO DE VARIÁVEIS - ENEM 2016,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NOME DA VARIÁVEL,Descrição,Variáveis Categóricas,NaN,Tamanho,Tipo
2,NaN,NaN,Categoria,Descrição,NaN,NaN
3,DADOS DO PARTICIPANTE,NaN,NaN,NaN,NaN,NaN
4,NU_INSCRICAO,Número de inscrição1,NaN,NaN,12,Numérica
